# **DIP（Deep Image Prior）**

- 由 Dmitry Ulyanov 等人在 2018 年提出的一種影像處理方法。它提出一個革命性的觀點：**不需要大量訓練資料，只用一個隨機初始化的 CNN，本身就能作為圖像的「先驗」知識**，用來解決圖像重建問題（如去雜訊、超解析、影像修復等）。

- **DIP = 不用訓練資料的影像重建方法，只靠一個神經網路的結構本身來恢復圖像。**

- 傳統方法會學習大量圖片分布，建立明確的「先驗」（如 GAN、DDPM），而 **DIP 的大膽想法是：CNN 的結構本身就是一種 implicit prior**，代表**一個隨機初始化的 CNN，天生就傾向於生成自然圖片的統計特性（如連續、平滑、局部結構），而不是雜訊。**

| 名稱 | Deep Image Prior (DIP) |
|------|------------------------|
| 發表年份 | 2018 |
| 作者 | Ulyanov, Vedaldi, Lempitsky |
| 核心想法 | CNN 結構本身提供影像先驗，不需大數據 |
| 特點 | 單圖像訓練，無需資料集 |
| 關鍵技術 | Random CNN + Loss + Early stopping |


### **核心原理**

- CNN 對雜訊的表現力 **遠小於** 對自然圖像的表現力。
- 隨機初始化的 CNN 一開始擅長產生平滑與結構化的模式，而不是雜訊。
- 所以，只要不訓練太久，它會先學會圖片結構，**還沒來得及學會雜訊就停下來**，就能恢復清晰的圖像。
- **Early Stopping 是關鍵**：需要觀察生成圖的變化，挑選最佳時機
    - 太早停：圖像還沒還原好
    - 太晚停：CNN 開始「記住」雜訊，導致 overfitting


### **目標：還原一張 degraded image（受損圖像）**

1. 建立一個隨機初始化的 CNN（例如 U-Net）。
2. 給這個網路一個隨機向量（如噪音）作為輸入。
3. 設定一個 loss function，度量輸出和原始損壞圖片的差異。
4. 不斷更新 CNN 的權重，使得輸出接近受損圖像，但不 overfitting 雜訊。


### **應用**：

1. **圖像去雜訊（Denoising）**
    - 給一張含雜訊的圖片作為目標
    - 使用 CNN 輸出逐漸貼近原圖
    - 關鍵：在模型開始 overfitting 雜訊前停止訓練（early stopping）

2. **圖像修補（Inpainting）**
    - 把部分遮住的圖片當作目標
    - Loss fn 只對「未遮住」的部分做誤差計算
    - CNN 會自動生成符合結構的補足圖像

3. **圖像超解析（Super-resolution）**
    - 低解析度圖片作為目標
    - CNN 預測高解析度結果並與低解析縮小版本比較

### **視覺化過程**

```plaintext
[隨機向量 z]
      ↓
CNN (隨機初始化)
      ↓
  產生圖片 f_θ(z)
      ↓
與受損圖像比較 loss
      ↓
更新 θ (gradient descent)
      ↓
重複多次 → 重建原圖
```


### **比較其他方法**：

| 特性 | DIP | GAN | DDPM |
|------|-----|-----|------|
| 是否需大量資料 | ✘ 不需要 | ✔ 需要 | ✔ 需要 |
| 是否訓練整個模型 | ✔ 每張圖都訓練一次 | ✔ 訓練一次可用於多圖 | ✔ 同上 |
| 圖像品質 | 高（需控制早停） | 高（但易不穩） | 極高（但慢） |
| 適用於 | 圖像復原 | 圖像生成 | 圖像生成 |


| 概念 | 說明 |
|------|------|
| U-Net | 一種 U 型 CNN，包含對稱的 Encoder 與 Decoder |
| Encoder | 負責特徵萃取（Down-sampeling） |
| Decoder | 負責圖像恢復（Up-Sampeling） |
| Skip Connection | 將 Encoder 特徵傳遞到 Decoder，保留圖像細節 |
| 應用 | 醫學影像分割、DIP、DDPM、Stable Diffusion 等 |


## **U-Net**

- 一種特殊設計的 CNN 架構，最初由 Olaf Ronneberger 等人在 2015 年為醫學影像分割任務所提出，但後來廣泛應用於各種影像處理任務（如：DIP、DDPM、圖像修補、超解析等）。

- **特色**
1. 它的名字來自於其**對稱的 U 型結構**
2. **U-Net = Down-sample + Up-sample + Skip Connections**
3. 在 U-Net 中，每次 Encoder 的輸出，都會跳過中間 Bottleneck，直接接到對應 Decoder 層。

```plaintext
輸入圖像
   ↓
[Encoder]（逐步 Down-sampling）
   ↓
    ──────⟶ Bottleneck ⟶
   ↑                          
[Decoder]（逐步 Up-sampling）
   ↑
輸出圖像
```

- **Encoder - 萃取高層次特徵**，透過：
    - 卷積層（Convolution）
    - ReLU acti_fn
    - 最大池化（Max Pooling）
    - 逐步**降低空間解析度**，但增加特徵 channel 數量

- **Decoder - 將特徵圖恢復回原始尺寸**，透過：
    - 反卷積（Transposed Convolution）或上採樣（Up-sampling）
    - 卷積層（Convolution）
    - 其任務是**重建原始影像的空間資訊**

## **Skip Connections -> U-Net 最關鍵的創新方法**

- 在 Encoder 的每一層輸出，**會直接連接到對應的 Decoder 層**，通常是**將 feature maps 做 concat（拼接）** 或加法運算，這樣做的目的是讓 Decoder：
    1. 不只是**從「抽象特徵」重建圖片**
    2. 還能**結合「原始的細節資訊」，讓輸出更清晰、更準確**

- **為什麼需要它？**：在壓縮的過程中，我們會丟失很多「低層次資訊」（如邊緣、細節等），如果沒有 skip connection，就很難完全恢復這些細節 -> **跳過中間 Bottleneck**

- **Skip connection 解決了這個問題：**
    - 保留了原圖的空間資訊
    - 加速訓練與收斂
    - 防止深層網路中的梯度消失（類似於 ResNet 的思想）

```plaintext
Input
  │
[Encoder1] ─────────────┐
  ↓                     │
[Encoder2] ───────┐     │
  ↓               ↓     ↓
[Bottleneck]      ↑     ↑
  ↓               │     │
[Decoder2] ◄──────┘     │
  ↓                     │
[Decoder1] ◄────────────┘
  ↓
Output
```

| 模型 | 特點 | 是否使用 Skip Connection |
|------|------|----------------------------|
| CNN | 特徵萃取 | ✘ 無 |
| ResNet | 深度學習中防止梯度消失 |  ✔（加法） |
| U-Net | 影像重建與分割 | ✔（拼接） |
| DIP | 使用 U-Net 結構做去雜訊 | ✔ |
| DDPM | 通常用 U-Net 當作噪音預測器 | ✔ |


In [ ]:
class DoubleConv(nn.Module):
    """(Conv => BN => ReLU) * 2
    U-Net 的基本 building block，對輸入的 feature 進行初步的特徵提取和轉換，改變 C 並引入非線性"""
    def __init__(self, in_channels, out_channels):
    def forward(self, x):

class Down(nn.Module):
    """Downscaling with maxpool then double conv ->  U-Net 的 encoder"""
    def __init__(self, in_channels, out_channels):
    def forward(self, x):

class Up(nn.Module):
    """Upscaling then double conv ->  U-Net 的 decoder"""
    def __init__(self, in_channels, out_channels, bilinear=True):
        '''
        scale_factor=2：
            - 如果輸入的尺寸是 (batch_size, channels, height, width)
            - 那麼輸出的尺寸將是 (batch_size, channels, 2*height, 2*width)

        align_corners：決定在調整尺寸時是否對齊角點像素
            - True: 輸入和輸出張量的角點像素將被對齊，並在調整尺寸時被保留，在某些情況下可以提供更精確的幾何對應
            - False (預設值): 角點像素可能不會完全對齊。
        在 mode='bilinear' 或 'bicubic' 模式下，建議將 align_corners 設置為 True，以獲得更一致的結果，尤其是在進行多個連續的上採樣或下採樣操作時
        '''

    def forward(self, x1, x2):
        '''
        torch.cat -> Skip-connection：
        將 Up-Sampeling 後的特徵圖 (x1) 與來自 Encoder 路徑中對應層的、具有更高分辨率的特徵圖 (x2) 在通道維度上連接起來
        這樣 Decoder 就能從 Encoder 中提取細節信息，有助於生成更精細的輸出
        '''

class OutConv(nn.Module):
    """U-Net 的輸出層 -> 將 U-Net 最終的特徵表示映射到所需的輸出圖像空間"""
    def __init__(self, in_channels, out_channels):
    def forward(self, x):


## **Down-sampling**

- 將原始影像或特徵圖的空間解析度 **縮小**（例如從 256×256 降為 128×128），以便：
    - 降低計算成本
    - 萃取**更抽象**的特徵

- **目標**：學會圖像的高階語義（what is it?），而非低階細節（where is it?）

### **常見的 Down-sampling 方法：**

| 方法 | 說明 |
|------|------|
| **Max Pooling** | 每個區塊取最大值（常用於 CNN） |
| **Average Pooling** | 每個區塊取平均值 |
| **Strided Convolution** | 使用步長 > 1 的卷積自動達到下採樣效果 |
| **Resize + Convolution** | 先縮圖再卷積（較少見） |

- 以 Max Pooling 為例，對 4×4 特徵圖使用 2×2 視窗進行下採樣：

```plaintext
原圖：          ↓ MaxPool 2x2
[1 2 5 6]       [5 8]
[4 5 8 7]  →    [9 7]
[9 2 3 1]
[3 9 7 0]
```

## **Up-sampling**

- 將特徵圖的空間解析度 **放大**（例如從 64×64 變成 128×128），**使最終輸出符合原始圖像大小**。

- **目標**：恢復圖像的細節與空間分布，用於圖像生成或復原

### **常見的 Up-sampling 方法：**

| 方法 | 說明 |
|------|------|
| **Nearest Neighbor** | 每個像素複製填滿更大的區塊（快速但粗糙） |
| **Bilinear / Bicubic Interpolation** | 根據鄰近像素做插值（平滑） |
| **Transposed Convolution**（Deconv） | 卷積的反操作，會學習上採樣的權重（常見於 U-Net） |
| **Pixel Shuffle**（亞像素卷積） | 將通道資訊轉回空間尺寸（用於超解析） |

- 以 Nearest Neighbor 為例，對 2×2 特徵圖放大至 4×4：

```plaintext
原圖：         上採樣結果：
[1 2]          [1 1 2 2]
[3 4]     →    [1 1 2 2]
               [3 3 4 4]
               [3 3 4 4]
```

| 名稱 | Down-sampling | Up-sampling |
|------|----------------|-------------|
| 目標 | 壓縮資訊、抽象理解       | 還原圖像、精細輸出       |
| 功能 | 降低尺寸、萃取語義 | 增加尺寸、恢復細節 |
| 方法 | MaxPool、Strided Conv | Nearest、Bilinear、Transposed Conv |
| 應用 | 特徵萃取、分類前處理 | 圖像恢復、影像生成 |
| 缺點 | 可能丟失細節 | 模糊、格狀 artifacts |
| 使用場景 | CNN Encoder、ResNet 等   | U-Net Decoder、GAN、VAE |


### **與 Skip Connection 的搭配**

- U-Net 就是透過下採樣學到抽象語義，再透過上採樣還原圖片，並搭配 Skip Connection 將低層細節直接傳遞回來，達到又準又細緻的輸出。

```plaintext
[Input]
   ↓
Down-sampling (MaxPool / Strided Conv)
   ↓
[抽象表示] ← Skip Connection ← Encoder features
   ↓
Up-sampling (Transposed Conv)
   ↓
[Output]
```

In [ ]:
class UNet(nn.Module):
    """UNet architecture for DIP"""
    def __init__(self, input_channels, output_channels, hidden_size, bilinear=True):
        '''
        bilinear=True：決定在 Up-Sampeling 時使用哪種插值方法
            - True: 使用雙線性插值 (bilinear interpolation) 一種平滑的 Up-Sampeling
            - False: 使用轉置卷積 (transposed convolution) 轉置卷積可讓網路學習如何進行 Up-Sampeling
        '''
    def forward(self, x):


In [ ]:
class DeepImagePrior(nn.Module):
    """Deep Image Prior model wrapper"""
    def __init__(self, args):
    def forward(self, noise):
        """通過神經網路將 noise 轉換為 image"""

    def get_random_input(self, target_shape, device='cuda'):
        """
        Generate random input noise for the DIP model
        根據 target_shape 反推 input noise size
        假設每層 UP-Sampeling x2，則 input noise size 為 H/(2^layers), W/(2^layers)
        """

    def generate_prior(self, target_image, iterations, reg_noise_std, device='cuda'):
        """Generate Prior for DIP-Guided DDPM"""
